In [1]:
from sklearn.svm import SVC

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
import pandas as panda

In [4]:
import numpy as np

In [26]:
import matplotlib.pyplot as plt
import seaborn

In [27]:
df = panda.read_csv("NAS.csv")

In [28]:
df.Close.dropna()

0       19.139799
1       18.454201
2       18.054300
3       17.425800
7       17.254400
          ...    
4248     0.676800
4249     0.600600
4250     0.626000
4251     0.605000
4252     0.610600
Name: Close, Length: 4218, dtype: float64

In [29]:
df = df.dropna()

In [30]:
y = np.where(df['Close'].shift(-1) > df['Close'],1,-1)

In [31]:
df['Open-Close'] = df.Open - df.Close


In [32]:
df['High-Low'] = df.High - df.Low
X = df[['Open-Close','High-Low']]


In [33]:
split_percentage = 0.8
split = int(split_percentage*len(df))

In [34]:
X_train = X[:split]
y_train = y[:split]

In [35]:
X_test = X[split:]
y_test = y[split:]

In [36]:
cls = SVC().fit(X_train, y_train)

In [37]:
accuracy_train = accuracy_score(y_train, cls.predict(X_train))

In [38]:
accuracy_test = accuracy_score(y_test, cls.predict(X_test))

In [39]:
print('\nTrain Accuracy:{: .2f}%'.format(accuracy_train*100))
print('Test Accuracy:{: .2f}%'.format(accuracy_test*100))


Train Accuracy: 54.80%
Test Accuracy: 54.74%


In [40]:
df['Predicted_Signal'] = cls.predict(X)

In [41]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Open-Close,High-Low,Predicted_Signal
0,2003-12-18,19.482599,19.596901,19.025499,19.139799,19.139799,4978496.0,0.342800,0.571402,-1
1,2003-12-19,19.368299,19.425501,18.282801,18.454201,18.454201,1410901.0,0.914098,1.142700,-1
2,2003-12-22,18.739901,18.739901,17.997101,18.054300,18.054300,137047.0,0.685601,0.742800,-1
3,2003-12-23,17.997101,17.997101,17.368700,17.425800,17.425800,229418.0,0.571301,0.628401,-1
7,2003-12-29,17.425800,17.425800,16.854500,17.254400,17.254400,196206.0,0.171400,0.571300,-1
...,...,...,...,...,...,...,...,...,...,...
4248,2020-10-12,0.709000,0.710000,0.650000,0.676800,0.676800,48320475.0,0.032200,0.060000,-1
4249,2020-10-13,0.676800,0.676800,0.600000,0.600600,0.600600,47786200.0,0.076200,0.076800,-1
4250,2020-10-14,0.601000,0.640000,0.596200,0.626000,0.626000,37534949.0,-0.025000,0.043800,-1
4251,2020-10-15,0.626000,0.626000,0.585000,0.605000,0.605000,26737615.0,0.021000,0.041000,-1


In [42]:
df['Return'] = np.log(df.Close.shift(-1) / df.Close)*100
df['Strategy_Return'] = df.Return * df.Predicted_Signal

In [43]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Open-Close,High-Low,Predicted_Signal,Return,Strategy_Return
0,2003-12-18,19.482599,19.596901,19.025499,19.139799,19.139799,4978496.0,0.342800,0.571402,-1,-3.647784,3.647784
1,2003-12-19,19.368299,19.425501,18.282801,18.454201,18.454201,1410901.0,0.914098,1.142700,-1,-2.190816,2.190816
2,2003-12-22,18.739901,18.739901,17.997101,18.054300,18.054300,137047.0,0.685601,0.742800,-1,-3.543202,3.543202
3,2003-12-23,17.997101,17.997101,17.368700,17.425800,17.425800,229418.0,0.571301,0.628401,-1,-0.988468,0.988468
7,2003-12-29,17.425800,17.425800,16.854500,17.254400,17.254400,196206.0,0.171400,0.571300,-1,2.614696,-2.614696
...,...,...,...,...,...,...,...,...,...,...,...,...
4248,2020-10-12,0.709000,0.710000,0.650000,0.676800,0.676800,48320475.0,0.032200,0.060000,-1,-11.944665,11.944665
4249,2020-10-13,0.676800,0.676800,0.600000,0.600600,0.600600,47786200.0,0.076200,0.076800,-1,4.142122,-4.142122
4250,2020-10-14,0.601000,0.640000,0.596200,0.626000,0.626000,37534949.0,-0.025000,0.043800,-1,-3.412191,3.412191
4251,2020-10-15,0.626000,0.626000,0.585000,0.605000,0.605000,26737615.0,0.021000,0.041000,-1,0.921362,-0.921362


In [44]:
df = df.dropna()

In [45]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Open-Close,High-Low,Predicted_Signal,Return,Strategy_Return
0,2003-12-18,19.482599,19.596901,19.025499,19.139799,19.139799,4978496.0,0.342800,0.571402,-1,-3.647784,3.647784
1,2003-12-19,19.368299,19.425501,18.282801,18.454201,18.454201,1410901.0,0.914098,1.142700,-1,-2.190816,2.190816
2,2003-12-22,18.739901,18.739901,17.997101,18.054300,18.054300,137047.0,0.685601,0.742800,-1,-3.543202,3.543202
3,2003-12-23,17.997101,17.997101,17.368700,17.425800,17.425800,229418.0,0.571301,0.628401,-1,-0.988468,0.988468
7,2003-12-29,17.425800,17.425800,16.854500,17.254400,17.254400,196206.0,0.171400,0.571300,-1,2.614696,-2.614696
...,...,...,...,...,...,...,...,...,...,...,...,...
4247,2020-10-09,0.800000,0.840000,0.644200,0.696400,0.696400,205161838.0,0.103600,0.195800,-1,-2.854840,2.854840
4248,2020-10-12,0.709000,0.710000,0.650000,0.676800,0.676800,48320475.0,0.032200,0.060000,-1,-11.944665,11.944665
4249,2020-10-13,0.676800,0.676800,0.600000,0.600600,0.600600,47786200.0,0.076200,0.076800,-1,4.142122,-4.142122
4250,2020-10-14,0.601000,0.640000,0.596200,0.626000,0.626000,37534949.0,-0.025000,0.043800,-1,-3.412191,3.412191
